In [4]:
# This is just a sample representation of the queries

In [5]:
# creating a dictionary for the stop words

stop_words={}
with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/Urdu stopwords.txt','r') as f:
    for line in f:
        stop_words[line.strip()]=1

In [6]:
queries = [
    'مجرم کو پھانسی دے دی گئی',
    'احتجاجی  دھرنے, جلوس اور ریلیاں',
    'انسداد تمباکو نوشی کے اقدامات اور آگاہی',
    'وکلاء کی ہڑتال اور ریلیاں',
    'خفیہ ایجنسیاں اور جاسوس',
    'احتساب عدالت کے مقدمات',
    'بچوں کے خلاف تشدد اور اغوا',
    'عورتوں پر ظلم و تشدد , قتل اور اغوا',
    'انسانوں کی اسمگلنگ اور انسداد انسانی سمگلنگ کے اقدامات',
    'مزدور طبقے کی خوشحالی کے اقدامات'
]

# converting to a dictionary with keys as index and values as queries
queries = {index: query for index, query in enumerate(queries)}
queries

# removing the stop words from the queries
for q_id, query in queries.items():
    query = query.split()
    query = [word for word in query if word not in stop_words]
    
    # removing punctuation
    query = [word.strip(',?') for word in query]

    # removing empty strings
    query = [word for word in query if word != '']
    
    queries[q_id] = query

queries

{0: ['مجرم', 'پھانسی'],
 1: ['احتجاجی', 'دھرنے', 'جلوس', 'ریلیاں'],
 2: ['انسداد', 'تمباکو', 'نوشی', 'اقدامات', 'آگاہی'],
 3: ['وکلاء', 'ہڑتال', 'ریلیاں'],
 4: ['خفیہ', 'ایجنسیاں', 'جاسوس'],
 5: ['احتساب', 'عدالت', 'مقدمات'],
 6: ['بچوں', 'خلاف', 'تشدد', 'اغوا'],
 7: ['عورتوں', 'ظلم', 'تشدد', 'قتل', 'اغوا'],
 8: ['انسانوں', 'اسمگلنگ', 'انسداد', 'انسانی', 'سمگلنگ', 'اقدامات'],
 9: ['مزدور', 'طبقے', 'خوشحالی', 'اقدامات']}

##### Now that we have queries in a dictionary, we can iterate through them and implement algorithms

In [8]:
# creating a hash table from the doc_index file as follows:
# key: doc_id&term_id
# value: term frequency

hashmap = {}
with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/doc_index.txt') as f:
    for line in f:
        line = line.strip().split()
        hashmap [line[0] + '&' + line[1]] = int(len(line[2:]))

In [17]:
# creating a list of all the documents
with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/docids.txt') as f:
    docs = [int(line.strip().split('\t')[-1]) for line in f.readlines()]
total_docs = len(docs)

5771

In [10]:
# creating a dictionary for all the term ids
term_ids = {}
with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/termids.txt') as f:
    for line in f:
        line = line.strip().split('\t')
        term_ids[line[1]] = int(line[0])

In [14]:
# getting IDF for each term from term_info.txt
term_idf = {}
with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/term_info.txt') as f:
    for line in f:
        line = line.strip().split('\t')
        term_idf[int(line[0])] = int(line[-1])

{1: 3281,
 2: 626,
 3: 121,
 4: 630,
 5: 470,
 6: 237,
 7: 320,
 8: 836,
 9: 2356,
 10: 1157,
 11: 107,
 12: 3832,
 13: 502,
 14: 48,
 15: 38,
 16: 115,
 17: 367,
 18: 1091,
 19: 490,
 20: 268,
 21: 284,
 22: 246,
 23: 41,
 24: 2551,
 25: 337,
 26: 1024,
 27: 132,
 28: 1242,
 29: 1250,
 30: 10,
 31: 1264,
 32: 115,
 33: 89,
 34: 291,
 35: 2726,
 36: 1119,
 37: 60,
 38: 78,
 39: 35,
 40: 452,
 41: 2140,
 42: 510,
 43: 989,
 44: 303,
 45: 34,
 46: 838,
 47: 391,
 48: 584,
 49: 255,
 50: 3,
 51: 3,
 52: 460,
 53: 1156,
 54: 3515,
 55: 2861,
 56: 548,
 57: 1960,
 58: 310,
 59: 396,
 60: 152,
 61: 286,
 62: 159,
 63: 23,
 64: 5793,
 65: 94,
 66: 45,
 67: 144,
 68: 2716,
 69: 200,
 70: 69,
 71: 2,
 72: 508,
 73: 1363,
 74: 20,
 75: 461,
 76: 270,
 77: 81,
 78: 454,
 79: 5,
 80: 8,
 81: 143,
 82: 705,
 83: 1551,
 84: 1961,
 85: 90,
 86: 2698,
 87: 5500,
 88: 918,
 89: 1742,
 90: 38,
 91: 536,
 92: 933,
 93: 1705,
 94: 786,
 95: 83,
 96: 1611,
 97: 880,
 98: 803,
 99: 471,
 100: 5,
 101: 4,
 1

In [16]:
# function to calculate the idf of a term
def get_idf(term):
    import math
    try:
        result = term_idf[term_ids[term]]
    except:
        result = 0
    # calculating the idf
    if result == 0:
        idf = 0
    else:
        idf = math.log(total_docs / result)
    return idf
# creating the query_idf dictionary
query_idf = {}
for q_id, query in queries.items():
    for word in query:
        query_idf[word] = get_idf(word)

query_idf

{'مجرم': 1.9747397076426072,
 'پھانسی': 1.905996555223004,
 'احتجاجی': 1.6113458134551297,
 'دھرنے': 2.785669923858936,
 'جلوس': 2.8985492719307895,
 'ریلیاں': 2.6867910428417052,
 'انسداد': 2.699595315087693,
 'تمباکو': 2.440010484611227,
 'نوشی': 2.354325367762951,
 'اقدامات': 1.9391749539203227,
 'آگاہی': 4.085889676207583,
 'وکلاء': 1.708828490312055,
 'ہڑتال': 2.005160304343319,
 'خفیہ': 2.277094019826961,
 'ایجنسیاں': 3.61717553779172,
 'جاسوس': 2.7689564428851954,
 'احتساب': 1.862660241736036,
 'عدالت': 0.8405626652522132,
 'مقدمات': 2.05937053598209,
 'بچوں': 1.1818658291430917,
 'خلاف': 0.47441466048488357,
 'تشدد': 1.4943346805773283,
 'اغوا': 2.2373536911774465,
 'عورتوں': 1.9923724062935633,
 'ظلم': 2.615595340674955,
 'قتل': 0.9795016531746074,
 'انسانوں': 4.0162097555695935,
 'اسمگلنگ': 2.755238806656396,
 'انسانی': 1.5369278695063593,
 'سمگلنگ': 3.3230625750096485,
 'مزدور': 2.3764664936401645,
 'طبقے': 3.392742495647638,
 'خوشحالی': 3.524802217660705}

In [ ]:
# getting the tf-idf for each complete query
def get_tf_idf(query, doc_id):
    tf_idf = 0
    for term in query:
        try:
            tf = hashmap[str(doc_id) + '&' + str(term_ids[term])]
            tf_idf += tf * query_idf[term]
        except:
            pass
    return tf_idf

In [ ]:
import time
import pandas as pd

# function to give a ranked dataframe for a given query
def get_ranked_df(query,qid):
    df = pd.DataFrame(columns=['query_id', 'doc_id', 'tf-idf', 'time'])
    for doc_id in docs:
        start = time.time()
        tf_idf = get_tf_idf(query, doc_id)
        end = time.time()
        
        # adding the data to the dataframe by concatenating
        df = pd.concat([df, pd.DataFrame([[qid, doc_id, tf_idf, end-start]], columns=['query_id', 'doc_id', 'tf-idf', 'time'])])
    print(query)
    
    # sort by query_id and tf-idf
    df.sort_values(by=['tf-idf'], ascending=False, inplace=True)

    # add a rank column
    df['rank'] = df.groupby('query_id').cumcount() + 1
    return df

In [ ]:
for q_id, query in queries.items():
    df = get_ranked_df(query, q_id)
    print(df.head(5),"\n")

In [4]:
# creating a standalone function to get the ranked dataframe for all the queries
def get_ranked_tf_idf():
    
    # -------------------------------------
    # creating a dictionary for the stop words
    stop_words={}
    with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/Urdu stopwords.txt','r') as f:
        for line in f:
            stop_words[line.strip()]=1

    # -------------------------------------
    queries = [
        'مجرم کو پھانسی دے دی گئی',
        'احتجاجی  دھرنے, جلوس اور ریلیاں',
        'انسداد تمباکو نوشی کے اقدامات اور آگاہی',
        'وکلاء کی ہڑتال اور ریلیاں',
        'خفیہ ایجنسیاں اور جاسوس',
        'احتساب عدالت کے مقدمات',
        'بچوں کے خلاف تشدد اور اغوا',
        'عورتوں پر ظلم و تشدد , قتل اور اغوا',
        'انسانوں کی اسمگلنگ اور انسداد انسانی سمگلنگ کے اقدامات',
        'مزدور طبقے کی خوشحالی کے اقدامات'
    ]

    # converting to a dictionary with keys as index and values as queries
    queries = {index: query for index, query in enumerate(queries)}
    queries

    # removing the stop words from the queries
    for q_id, query in queries.items():
        query = query.split()
        query = [word for word in query if word not in stop_words]
        
        # removing punctuation
        query = [word.strip(',?') for word in query]

        # removing empty strings
        query = [word for word in query if word != '']
        
        queries[q_id] = query

    # -------------------------------------
    # creating a hash table from the doc_index file as follows:
    # key: doc_id&term_id
    # value: term frequency

    hashmap = {}
    with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/doc_index.txt') as f:
        for line in f:
            line = line.strip().split()
            hashmap [line[0] + '&' + line[1]] = int(len(line[2:]))

    # -------------------------------------
    # creating a list of all the documents
    with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/docids.txt') as f:
        docs = [int(line.strip().split('\t')[-1]) for line in f.readlines()]
    total_docs = len(docs)

    # -------------------------------------
    # creating a dictionary for all the term ids
    term_ids = {}
    with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/termids.txt') as f:
        for line in f:
            line = line.strip().split('\t')
            term_ids[line[1]] = int(line[0])
    
    # -------------------------------------
    # getting IDF for each term from term_info.txt
    term_idf = {}
    with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/term_info.txt') as f:
        for line in f:
            line = line.strip().split('\t')
            term_idf[int(line[0])] = int(line[-1])
    
    # -------------------------------------
    # function to calculate the idf of a term
    def get_idf(term):
        import math
        try:
            result = term_idf[term_ids[term]]
        except:
            result = 0
        # calculating the idf
        if result == 0:
            idf = 0
        else:
            idf = math.log(total_docs / result)
        return idf
    # creating the query_idf dictionary
    query_idf = {}
    for q_id, query in queries.items():
        for word in query:
            query_idf[word] = get_idf(word)    

    # -------------------------------------
    # getting teh tf-idf for each complet query
    def get_tf_idf(query, doc_id):
        tf_idf = 0
        for term in query:
            try:
                tf = hashmap[str(doc_id) + '&' + str(term_ids[term])]
                tf_idf += tf * query_idf[term]
            except:
                pass
        return tf_idf
    
    # -------------------------------------
    import time
    import pandas as pd

    # function to give a ranked dataframe for a given query
    def get_ranked_df(query,qid):
        df = pd.DataFrame(columns=['query_id', 'doc_id', 'tf-idf', 'time'])
        for doc_id in docs:
            start = time.time()
            tf_idf = get_tf_idf(query, doc_id)
            end = time.time()
            
            # adding the data to the dataframe by concatenating
            df = pd.concat([df, pd.DataFrame([[qid, doc_id, tf_idf, end-start]], columns=['query_id', 'doc_id', 'tf-idf', 'time'])])
        print(query)
        
        # sort by query_id and tf-idf
        df.sort_values(by=['tf-idf'], ascending=False, inplace=True)

        # add a rank column
        df['rank'] = df.groupby('query_id').cumcount() + 1
        return df
    
    # -------------------------------------
    for q_id, query in queries.items():
        df = get_ranked_df(query, q_id)
        print(df.head(5),"\n")

In [5]:
get_ranked_tf_idf()

['مجرم', 'پھانسی']
  query_id doc_id     tf-idf      time  rank
0        0   1091  54.399051  0.000004     1
0        0   1131  38.738619  0.000004     2
0        0   1082  32.951887  0.000004     3
0        0   1322  30.839661  0.000018     4
0        0   1364  29.139894  0.000010     5 

['احتجاجی', 'دھرنے', 'جلوس', 'ریلیاں']
  query_id doc_id     tf-idf      time  rank
0        1   2788  60.869535  0.000006     1
0        1   2887   40.57969  0.000038     2
0        1   2824  37.681141  0.000010     3
0        1   2934  20.289845  0.000006     4
0        1    244  20.289845  0.000006     5 

['انسداد', 'تمباکو', 'نوشی', 'اقدامات', 'آگاہی']
  query_id doc_id     tf-idf      time  rank
0        2   2292  21.137895  0.000007     1
0        2   2638  20.075922  0.000010     2
0        2   2715  18.897167  0.000008     3
0        2   1707  15.855486  0.000006     4
0        2   1529  15.095065  0.000007     5 

['وکلاء', 'ہڑتال', 'ریلیاں']
  query_id doc_id     tf-idf      time  rank
0  

In [1]:
# applying the BM25 algorithm
def get_ranked_bm25():
        
        # -------------------------------------
        # creating a dictionary for the stop words
        stop_words={}
        with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/Urdu stopwords.txt','r') as f:
            for line in f:
                stop_words[line.strip()]=1
    
        # -------------------------------------
        queries = [
            'مجرم کو پھانسی دے دی گئی',
            'احتجاجی  دھرنے, جلوس اور ریلیاں',
            'انسداد تمباکو نوشی کے اقدامات اور آگاہی',
            'وکلاء کی ہڑتال اور ریلیاں',
            'خفیہ ایجنسیاں اور جاسوس',
            'احتساب عدالت کے مقدمات',
            'بچوں کے خلاف تشدد اور اغوا',
            'عورتوں پر ظلم و تشدد , قتل اور اغوا',
            'انسانوں کی اسمگلنگ اور انسداد انسانی سمگلنگ کے اقدامات',
            'مزدور طبقے کی خوشحالی کے اقدامات'
        ]
    
        # converting to a dictionary with keys as index and values as queries
        queries = {index: query for index, query in enumerate(queries)}
        queries
    
        # removing the stop words from the queries
        for q_id, query in queries.items():
            query = query.split()
            query = [word for word in query if word not in stop_words]
            
            # removing punctuation
            query = [word.strip(',?') for word in query]
    
            # removing empty strings
            query = [word for word in query if word != '']
            
            queries[q_id] = query
    
        # -------------------------------------
        # creating a hash table from the doc_index file as follows: 
        # key: doc_id&term_id
        # value: term frequency

        hashmap = {}
        with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/doc_index.txt') as f:
            for line in f:
                line = line.strip().split()
                hashmap [line[0] + '&' + line[1]] = int(len(line[2:]))

        # -------------------------------------
        # creating a list of all the documents
        with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/docids.txt') as f:
            docs = [int(line.strip().split('\t')[-1]) for line in f.readlines()]
        total_docs = len(docs)

        # -------------------------------------
        # creating a dictionary for all the term ids
        term_ids = {}
        with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/termids.txt') as f:
            for line in f:
                line = line.strip().split('\t')
                term_ids[line[1]] = int(line[0])

        # -------------------------------------
        # getting IDF for each term from term_info.txt
        term_idf = {}
        with open('/Users/carbon/Documents/Semester 6/Information Retreival/Assignments/Assignment 1/term_info.txt') as f:
            for line in f:
                line = line.strip().split('\t')
                term_idf[int(line[0])] = int(line[-1])

        # -------------------------------------
        # function to calculate the idf of a term
        def get_idf(term):
            import math
            try:
                result = term_idf[term_ids[term]]
            except:
                result = 0
            # calculating the idf
            if result == 0:
                idf = 0
            else:
                idf = math.log(total_docs / result)
            return idf
        
        # creating the query_idf dictionary
        query_idf = {}
        for q_id, query in queries.items():
            for word in query:
                query_idf[word] = get_idf(word)

        # -------------------------------------
        # getting the tf-idf for each complet query
        def get_tf_idf(query, doc_id):
            tf_idf = 0
            for term in query:
                try:
                    tf = hashmap[str(doc_id) + '&' + str(term_ids[term])]
                    tf_idf += tf * query_idf[term]
                except:
                    pass
            return tf_idf
        
        # -------------------------------------
        import time
        import pandas as pd

        # function to give a ranked dataframe for a given query
        def get_ranked_df(query,qid):
            df = pd.DataFrame(columns=['query_id', 'doc_id', 'bm25', 'time'])
            for doc_id in docs:
                start = time.time()
                bm25 = get_tf_idf(query, doc_id)
                end = time.time()
                
                # adding the data to the dataframe by concatenating
                df = pd.concat([df, pd.DataFrame([[qid, doc_id, bm25, end-start]], columns=['query_id', 'doc_id', 'bm25', 'time'])])
            print(query)
            
            # sort by query_id and bm25
            df.sort_values(by=['bm25'], ascending=False, inplace=True)

            # add a rank column
            df['rank'] = df.groupby('query_id').cumcount() + 1
            return df
        
        # -------------------------------------
        for q_id, query in queries.items():
            df = get_ranked_df(query, q_id)
            print(df.head(5),"\n")

get_ranked_bm25()

['مجرم', 'پھانسی']
  query_id doc_id       bm25      time  rank
0        0   1091  54.399051  0.000004     1
0        0   1131  38.738619  0.000019     2
0        0   1082  32.951887  0.000005     3
0        0   1322  30.839661  0.000005     4
0        0   1364  29.139894  0.000025     5 

['احتجاجی', 'دھرنے', 'جلوس', 'ریلیاں']
  query_id doc_id       bm25      time  rank
0        1   2788  60.869535  0.000041     1
0        1   2887   40.57969  0.000014     2
0        1   2824  37.681141  0.000020     3
0        1   2934  20.289845  0.000012     4
0        1    244  20.289845  0.000008     5 

['انسداد', 'تمباکو', 'نوشی', 'اقدامات', 'آگاہی']
  query_id doc_id       bm25      time  rank
0        2   2292  21.137895  0.000007     1
0        2   2638  20.075922  0.000013     2
0        2   2715  18.897167  0.000012     3
0        2   1707  15.855486  0.000025     4
0        2   1529  15.095065  0.000015     5 

['وکلاء', 'ہڑتال', 'ریلیاں']
  query_id doc_id       bm25      time  rank
0  

In [6]:
!python ass2.py --score TF-IDF

['مجرم', 'پھانسی']
  query_id doc_id     tf-idf      time  rank
0        0   1091  54.399051  0.000005     1
0        0   1131  38.738619  0.000007     2
0        0   1082  32.951887  0.000019     3
0        0   1322  30.839661  0.000004     4
0        0   1364  29.139894  0.000010     5 

['احتجاجی', 'دھرنے', 'جلوس', 'ریلیاں']
  query_id doc_id     tf-idf      time  rank
0        1   2788  60.869535  0.000006     1
0        1   2887   40.57969  0.000008     2
0        1   2824  37.681141  0.000006     3
0        1   2934  20.289845  0.000007     4
0        1    244  20.289845  0.000008     5 

['انسداد', 'تمباکو', 'نوشی', 'اقدامات', 'آگاہی']
  query_id doc_id     tf-idf      time  rank
0        2   2292  21.137895  0.000006     1
0        2   2638  20.075922  0.000005     2
0        2   2715  18.897167  0.000008     3
0        2   1707  15.855486  0.000006     4
0        2   1529  15.095065  0.000008     5 

['وکلاء', 'ہڑتال', 'ریلیاں']
  query_id doc_id     tf-idf      time  rank
0  